<a href="https://colab.research.google.com/github/demetriusohalloran-byte/DevOps/blob/main/Integrate_Traditional_Chatbot_with_AI_Service_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import sys, subprocess, importlib, math, re, datetime
def _ensure(pkg):
    try:
        importlib.import_module(pkg)
    except ImportError:
        print(f"Installing {pkg} ...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg, "-q"])
_ensure("vaderSentiment")

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
VADER = SentimentIntensityAnalyzer()

HELLO_RE = re.compile(r"\b(hi|hello|hey)\b", re.I)
TIME_RE  = re.compile(r"\b(time|what\s+time\s+is\s+it\??)\b", re.I)
HOW_RE   = re.compile(r"\bhow\s+are\s+you\??\b", re.I)
CAPS_RE  = re.compile(r"\b(capabilities|help|menu)\b", re.I)

HELP_TEXT = (
    "I can:\n"
    "- Greet you (say 'hello' or 'hi')\n"
    "- Tell the local time (say 'time' or 'what time is it?')\n"
    "- Answer 'how are you?'\n"
    "- Show this list (type 'capabilities' or 'help')\n"
    "- Analyze sentiment (type: sentiment: <your text>)\n"
    "- Handle malformed input politely\n"
    "- Exit cleanly (type 'exit' or 'quit')\n"
    "- Try a semantic FAQ assist on unknown questions\n"
)

def local_time_str() -> str:
    # Use datetime.datetime.now(datetime.timezone.utc) for a timezone-aware UTC datetime,
    # then convert to local timezone.
    now_local = datetime.datetime.now(datetime.timezone.utc).astimezone()
    return now_local.strftime("%I:%M:%S %p %Z")

def looks_malformed(s: str) -> bool:
    # 4+ repeated non-alphanumeric symbols in a row counts as malformed
    return bool(re.search(r"([^A-Za-z0-9\s])\1\1\1", s))


def analyze_sentiment(text: str):
    if not text.strip():
        return "neutral", {"positive": 0.0, "neutral": 1.0, "negative": 0.0}
    s = VADER.polarity_scores(text)
    comp = s["compound"]
    label = "positive" if comp >= 0.05 else "negative" if comp <= -0.05 else "neutral"
    return label, {"positive": float(s["pos"]), "neutral": float(s["neu"]), "negative": float(s["neg"])}


KB = [
    ("what can you do", "I list capabilities with 'help', greet, tell time, answer 'how are you', analyze sentiment, and exit."),
    ("how to see capabilities", "Type 'help' or 'capabilities' and I will show my abilities."),
    ("how to exit", "Type 'exit' or 'quit' to end the session cleanly."),
    ("how to check time", "Ask 'time' or 'what time is it?' and I will print the local time."),
    ("how to run sentiment", "Use 'sentiment: <your text>' and I will analyze the tone using a local model."),
    ("what is semantic search", "I compare your words with my FAQ articles using a bag of words and cosine similarity to find the closest answer."),
]

TOKEN_RE = re.compile(r"[A-Za-z0-9']+")

def _tokenize(text: str):
    return [t.lower() for t in TOKEN_RE.findall(text)]

def _tf_counts(tokens):
    d = {}
    for t in tokens:
        d[t] = d.get(t, 0) + 1
    return d

def _cosine(a: dict, b: dict):
    # simple cosine on term frequency vectors
    if not a or not b:
        return 0.0
    # dot
    dot = 0.0
    for k, v in a.items():
        if k in b:
            dot += v * b[k]
    # norms
    na = math.sqrt(sum(v*v for v in a.values()))
    nb = math.sqrt(sum(v*v for v in b.values()))
    if na == 0 or nb == 0:
        return 0.0
    return dot / (na * nb)

KB_VECS = [(q, a, _tf_counts(_tokenize(q))) for (q, a) in KB]

def semantic_faq(query: str, threshold: float = 0.22):
    qvec = _tf_counts(_tokenize(query))
    best = (None, None, 0.0)
    for q, a, v in KB_VECS:
        s = _cosine(qvec, v)
        if s > best[2]:
            best = (q, a, s)
    if best[2] >= threshold:
        return f"FAQ match: {best[1]} (score={best[2]:.2f})"
    return None


def handle_input(user_text: str) -> str:
    if not user_text or not user_text.strip():
        return "Please say something! Type 'help' to see what I can do."

    text = user_text.strip()

    # explicit sentiment command
    if text.lower().startswith("sentiment:"):
        payload = text.split(":", 1)[1].strip()
        label, scores = analyze_sentiment(payload)
        return (
            f"Sentiment \u2192 {label.capitalize()} "
            f"(pos={scores['positive']:.2f}, neu={scores['neutral']:.2f}, neg={scores['negative']:.2f})."
        )

    # traditional intents
    if CAPS_RE.search(text):
        return HELP_TEXT

    if HELLO_RE.search(text):
        return "Hello! How can I help you today?"

    if HOW_RE.search(text):
        return "I'm doing great, thank you! I'm here to help."

    if TIME_RE.search(text):
        return f"The current local time is {local_time_str()}."

    if looks_malformed(text):
        return "That looks a bit garbled. Try plain text or type 'help'."


    faq = semantic_faq(text)
    if faq:
        return faq


    label, scores = analyze_sentiment(text)
    return (
        "I don't know that one yet. Tip: type 'help' to see commands.\n"
        f"(Sentiment hint: {label}, pos={scores['positive']:.2f}, "
        f"neu={scores['neutral']:.2f}, neg={scores['negative']:.2f})"
    )

def chat():
    print("Welcome to the AI-assisted Traditional Chatbot!")
    print("Type 'help' for abilities, 'exit' to quit.\n")
    while True:
        try:
            user = input("You: ")
        except (KeyboardInterrupt, EOFError):
            print("\nBot: Goodbye!")
            break

        if not user:
            print("Bot: Please say something!\n")
            continue

        if user.strip().lower() in {"exit", "quit"}:
            print("Bot: Goodbye! Have a great day!\n")
            break

        print("Bot:", handle_input(user), "\n")


chat()


Welcome to the AI-assisted Traditional Chatbot!
Type 'help' for abilities, 'exit' to quit.

You: time
Bot: The current local time is 02:43:30 PM UTC. 


Bot: Goodbye!
